In [0]:
import requests
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType

headers = {
    'accept': 'application/json',
    'x-access-token-ws': 'd1bf4238656c2167c5997e4081b549eedd0a25622a18740d8ad1a8a31ee22b15'
}

# -------------------- EXTRAÇÃO ---------------------------------------
url = 'https://api.irancho.com.br/api/animal'
response = requests.get(url, headers=headers)
animals_list = response.json()
df_animal_pandas = pd.json_normalize(animals_list, sep='.')
print(f'animal {response.status_code}')

# ✅ CONVERTER TUDO PARA STRING NO PANDAS
df_animal_pandas = df_animal_pandas.astype(str)

colunas_usadas = ["dt_minima",
"idade_meses",
"gmd",
"nome",
"peso",
"publico",
"id_sexo",
"id_raca",
"id_lote",
"id_tipo_chifre",
"id_cor_pelagem",
"id_animal",
"id_venda_animal",
"id_fazenda",
"id_atividade",
"dt_ultima_movimentacao",
"dt_ultimo_aparte",
"dt_nascimento",
"ic_animal_ativo",
"id_padrao_racial",
"id_animal_fazenda",
"ic_animal_castrado",
"id_categoria_animal",
"id_situacao_vaca",
"id_brinco_eletronico",
"id_fazenda_subdivisao",
"id_rastreabilidade_manejo",
"id_rastreabilidade",
"id_animal_nascimento",
"id_animal_definitivo",
"no_animal",
"id_vaca",
"id_touro",
"dt_morte_animal",
"id_morte",
"id_tipo_parto",
"dt_ultima_pesagem",
"dt_criacao",
"sr_sui_abcz",
"dt_deletado",
"dt_desativado",
"idade_meses_desativacao",
"safe_info",
"venda",
"lote_atual",
"tipo_parto",
"tipo_chifre",
"pelagem",
"situacao_vaca",
"nascimento",
"carencia_abate",
"morte",
"historico_baia_giro",
"partidas",
"crias",
"partos_da_vaca",
"sexo.no_sexo",
"raca.no_raca",
"atividade_atual.no_animal_atividade",
"atividade_atual.id_animal_atividade",
"padrao_racial.no_padrao_racial",
"padrao_racial.id_padrao_racial",
"categoria_atual.no_categoria_animal",
"categoria_atual.id_categoria_animal",
"fazenda.nome",
"fazenda.id_fazenda",
"subdivisao.id_fazenda_subdivisao",
"detalhes.ps_nascimento",
"detalhes.ps_desmame",
"detalhes.id_animal",
"detalhes.de_animal_observacao",
"detalhes.dt_id_rastreabilidade",
"detalhes.dt_certificacao_rastreabilidade",
"detalhes.dt_compra_animal",
"detalhes.vr_compra_animal",
"detalhes.dt_venda_animal",
"detalhes.vr_venda_animal",
"detalhes.ps_frigorifico",
"detalhes.pct_rend_carcaca",
"detalhes.nu_ordem_nascimento",
"detalhes.ic_tamanho_nascimento",
"detalhes.ic_rejeitado",
"detalhes.ic_natimorto",
"detalhes.dt_desmame",
"detalhes.dt_vaca_primeiro_parto",
"detalhes.dt_vaca_ultimo_parto",
"detalhes.qt_vaca_partos",
"detalhes.altura_garupa",
"detalhes.id_resgistro_nascimento",
"detalhes.id_categoria_registro",
"detalhes.id_situacao_nascimento",
"detalhes.id_categoria_vaca",
"detalhes.id_contato_fazenda_origem",
"detalhes.id_contato_origem",
"detalhes.id_morte_causa",
"detalhes.id_fazenda_desativado",
"detalhes.id_subdivisao_desativado",
"detalhes.id_exposicao_reprodutiva",
"detalhes.a12",
"detalhes.sr_paint_cia",
"detalhes.id_assinatura",
"detalhes.id_animal_origem",
"detalhes.id_brinco_eletronico_anterior",
"detalhes.dt_troca_brinco_eletronico",
"detalhes.fk_id_responsavel_troca_brinco_eletronico",
"detalhes.dt_criacao",
"detalhes.dt_alteracao",
"detalhes.fk_id_contato_fazenda_origem",
"detalhes.fazenda_desativado",
"detalhes.subdivisao_desativado",
"detalhes.categoria_registro",
"detalhes.situacao_nascimento",
"detalhes.categoria_vaca",
"detalhes.origem",
"detalhes.contato_origem",
"detalhes.morte_causa",
"situacao_vaca.no_situacao_vaca",
"situacao_vaca.id_situacao_vaca"]

# Criar schema com tudo como String
schema = StructType([
    StructField(col, StringType(), True) 
    for col in colunas_usadas
])

df_animal = spark.createDataFrame(
    df_animal_pandas[colunas_usadas],  # passa só X, Y, Z
    schema=schema                      # schema com só essas colunas também
)
df_animal.write.mode("overwrite").saveAsTable("animal_raw")

print("✅ Tabela criada com sucesso!")

In [0]:
import requests
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType

headers = {
    'accept': 'application/json',
    'x-access-token-ws': 'd1bf4238656c2167c5997e4081b549eedd0a25622a18740d8ad1a8a31ee22b15'
}

# -------------------- EXTRAÇÃO ---------------------------------------
url = 'https://api.irancho.com.br/api/animal/pesagens'
response = requests.get(url, headers=headers)
pesagens_list = response.json()
df_pesagens_pandas = pd.json_normalize(pesagens_list, sep='.')
print(f'pesagens {response.status_code}')

# ✅ CONVERTER TUDO PARA STRING NO PANDAS
df_pesagens_pandas = df_pesagens_pandas.astype(str)

# Criar schema com tudo como String
schema = StructType([
    StructField(col, StringType(), True) 
    for col in df_pesagens_pandas.columns
])

df_pesagens = spark.createDataFrame(df_pesagens_pandas, schema=schema)
df_pesagens.write.mode("overwrite").saveAsTable("pesagens_raw")

print("✅ Tabela criada com sucesso!")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import (
    col,
    regexp_replace,
    schema_of_json,
    from_json,
    explode
)

# 1) --- Troca aspas simples por aspas duplas
df_clean = df_pesagens.withColumn(
    "rows_clean",
    regexp_replace(col("rows"), "'", '"')
)

# 2) --- Troca valores Python pelos correspondentes JSON
df_clean = (
    df_clean
    .withColumn("rows_clean", regexp_replace("rows_clean", "None", "null"))
    .withColumn("rows_clean", regexp_replace("rows_clean", "True", "true"))
    .withColumn("rows_clean", regexp_replace("rows_clean", "False", "false"))
)

# 3) --- Pegar um exemplo para inferir o schema JSON
sample_json = df_clean.select("rows_clean").first()[0]

# 4) --- Inferir automaticamente o schema JSON
json_schema = (
    spark.range(1)
         .select(schema_of_json(sample_json).alias("schema"))
         .first()["schema"]
)

# 5) --- Converter STRING → ARRAY<STRUCT>
df_array = df_clean.withColumn(
    "rows_array",
    from_json(col("rows_clean"), json_schema)
)

# 6) --- Explodir a lista
df_exploded = df_array.select(explode("rows_array").alias("row"))

# 7) --- Expandir os campos internos
df_final = df_exploded.select("row.*")

df_exploded = df_final.select(
    explode("pesagens").alias("pesagem")
)

df_pesagens_final=df_exploded.select("pesagem.*")

df_pesagens_final.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("pesagens_raw")



Definições das tabelas 


In [0]:
%sql
create or replace table WORKSPACE.IRANCHO.animal as /*FALTA RENOMEAR AS COLUNAS ANINHADAS E TIRAR AS COLUNAS INUTEIS*/
select * 
from animal_raw; 

create or replace table workspace.irancho.pesagens as 
  select * 
from pesagens_raw;